In [1]:
import time
import os
import cv2
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm

from models import Autoencoder
from loss import VGGPerceptualLoss

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torchvision.transforms as transforms

from torchvision.utils import make_grid

import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline

In [2]:
# TODO:
#     LR_SCHEDULE
#     NORMALIZation
#     vgg perceptual loss
#     visdom    

In [3]:
# from visdom import Visdom

# class VisdomEnv(object):
#     def __init__(self, env_name='main'):
#         self.viz = Visdom()
#         self.env = env_name

# class VisdomPlotter(VisdomEnv):
#     def __init__(self):
#         super(VisdomPlotter, self).__init__()
#         self.plots = {}
        
#     def plot(self, var_name, split_name, title_name, x, y):
#         if var_name not in self.plots:
#             self.plots[var_name] = self.viz.line(X=np.array([x,x]), Y=np.array([y,y]), env=self.env, opts=dict(
#                 legend=[split_name],
#                 title=title_name,
#                 xlabel='Epochs',
#                 ylabel=var_name
#             ))
#         else:
#             self.viz.line(X=np.array([x]), Y=np.array([y]), env=self.env, win=self.plots[var_name], name=split_name, update = 'append')
            
# class VisdomImage(VisdomEnv):  
#     def __init__(self):
#         super(VisdomImage, self).__init__()
        
#     def show(self, img, text=None):
#         self.viz.text(text)
#         self.viz.image(img)
        
# plotter = VisdomPlotter()
# imager = VisdomImage()

In [4]:
training_previews_folder = "./training_previews"
show_every = 500

if not os.path.exists(training_previews_folder):
    os.makedirs(training_previews_folder)

def training_window(batchA, batchB, modelA, modelB, epoch, iteration):
    show=True
        
    if show:
        if i % show_every == 0:
        
            if len(batchA) < 4 or len(batchB) < 4:
                raise ValueError("lenghts of batches has to be grater that 4.")

            batchA = batchA[:6]
            batchB = batchB[:6]

            A_actual = []
            A_recreated = []
            A_swapped = []
            for img in batchA:
                A_actual.append(img.permute(1,2,0).detach().cpu().numpy())

                img = img.unsqueeze(0)

                A_recreated.append(modelA(img).squeeze(0).permute(1,2,0).detach().cpu().numpy())
                A_swapped.append(modelB.decoder(modelA.encoder(img)).squeeze(0).permute(1,2,0).detach().cpu().numpy())

            B_actual = []
            B_recreated = []
            B_swapped = []
            for img in batchB:
                B_actual.append(img.permute(1,2,0).detach().cpu().numpy())

                img = img.unsqueeze(0)

                B_recreated.append(modelB(img).squeeze(0).permute(1,2,0).detach().cpu().numpy())
                B_swapped.append(modelA.decoder(modelB.encoder(img)).squeeze(0).permute(1,2,0).detach().cpu().numpy())

            col1 = np.vstack(A_actual)
            col2 = np.vstack(A_recreated)
            col3 = np.vstack(A_swapped)

            col4 = np.vstack(B_actual)
            col5 = np.vstack(B_recreated)
            col6 = np.vstack(B_swapped)

            window = np.hstack([col1, col2, col3, col4, col5, col6])
    #         window = cv2.resize(window, (800, 800), interpolation=cv2.INTER_AREA)

            cv2.imshow("Training window", cv2.cvtColor(window, cv2.COLOR_BGR2RGB))
#             imager.show(window)
    
            cv2.imwrite(f'{training_previews_folder}/preview_at_epoch{epoch}_iteration{iteration}.jpg', window) 
        
        if cv2.waitKey(33)==27:    # Esc key to stop
            show=False
# for i in range(20):
#     imgt = Image.open(fls[i])
#     cv2.imshow("preview some", cv2.cvtColor(np.array(imgt), cv2.COLOR_BGR2RGB))
#     time.sleep(1)
#     k = cv2.waitKey(33)
#     if k==27:    # Esc key to stop
#         break

In [5]:
class Faceset(Dataset):
    def __init__(self, root, transforms):
        
        self.transforms = transforms
        
        self.imgs_A = glob(f'{root}/A/*')
        self.imgs_B = glob(f'{root}/B/*')    
        
    def __len__(self):
        if len(self.imgs_A) > len(self.imgs_B):
            return len(self.imgs_B)
        else:
            return len(self.imgs_A)
    
    def __getitem__(self, idx):
        
        img_A = self.transforms(Image.open(self.imgs_A[idx]))
        img_B = self.transforms(Image.open(self.imgs_B[idx]))
        
        return img_A, img_B

In [6]:
epochs = 10000
change_lr_each = 2500 #epochs

In [7]:
transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
])

dataset = Faceset('./data/faces/', transforms)
loader = DataLoader(dataset,
                    batch_size=32,
                    shuffle=True,
                    num_workers=2)

In [8]:
len(dataset)

43443

In [9]:
modelA = Autoencoder().to(device)
modelB = Autoencoder().to(device)

criterion = VGGPerceptualLoss()#nn.MSELoss()

optimizerA = optim.Adam(modelA.parameters(), lr=5e-2)
optimizerB = optim.Adam(modelB.parameters(), lr=5e-2)

schedulerA = optim.lr_scheduler.StepLR(optimizerA, change_lr_each)
schedulerB = optim.lr_scheduler.StepLR(optimizerB, change_lr_each)

In [ ]:
load weights

In [ ]:
# plotA = []
# plotB = []

trained_models = "./models"

if not os.path.exists(trained_models):
    os.makedirs(trained_models)

for epoch in tqdm(range(epochs)):
    for i, (imgsA, imgsB) in enumerate(loader):
        imgsA = imgsA.to(device)
        imgsB = imgsB.to(device)
        
        optimizerA.zero_grad()
        optimizerB.zero_grad()
        
        outputsA = modelA(imgsA)
        outputsB = modelB(imgsB)
        
        lossA = criterion(outputsA, imgsA)
        lossB = criterion(outputsB, imgsB)        
        
        lossA.backward()
        lossB.backward()
        
        optimizerA.step()
        optimizerB.step()
        
        schedulerA.step()
        schedulerB.step()
        
        plotA.append(lossA.item())
        plotB.append(lossB.item())
#         plotter.plot('loss', 'val', 'Loss', epoch, [lossA.item(), lossB.item()])
                
        training_window(imgsA, imgsB, modelA, modelB, epoch, i)
            
    torch.save(modelA.state_dict(), f"{trained_models}/modelA_at_epoch_{epoch}.pth")
    torch.save(modelB.state_dict(), f"{trained_models}/modelB_at_epoch_{epoch}.pth")


  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
plt.plot(plotA)
plt.plot(plotB)
plt.legend(["Loss model A", "Loss model B"])